#Survival Analysis of SPIRE Luminosity Distribution

In this notebook I am going to characterize the SPIRE luminosity distributions for the BAT AGN. Each wavelength (250, 350, 500) will be analyzed as well as breaking up the sample into Seyfert types. I will use survival analysis to take into account all of the upper limits.

In [3]:
# Normal science modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [50]:
# Survival analysis package
import rpy2.robjects as robjects
import rpy2.robjects.vectors as vecs
from rpy2.robjects import Formula
from rpy2.robjects.packages import importr
import pandas.rpy.common as com
surv = importr('survival')

In [13]:
# Import the SPIRE data
bat_herschel = pd.read_csv('/Users/ttshimiz/Github/bat-data/bat_herschel.csv', index_col=0)
bat_info = pd.read_csv('/Users/ttshimiz/Github/bat-data/bat_info.csv', index_col=0)

In [39]:
# Calculate luminosities for each wavelength and each source
# L = 4*pi*D^2*F
l250 = 4*np.pi*(bat_info['Dist_[Mpc]']*10**6*3.09e18)**2*(3.0e10/250.0e-4)*bat_herschel['PSW']*10**(-23)
l350 = 4*np.pi*(bat_info['Dist_[Mpc]']*10**6*3.09e18)**2*(3.0e10/350.0e-4)*bat_herschel['PMW']*10**(-23)
l500 = 4*np.pi*(bat_info['Dist_[Mpc]']*10**6*3.09e18)**2*(3.0e10/500.0e-4)*bat_herschel['PLW']*10**(-23)

# Input the 5-sigma upper limits
l250[l250 == 0] = 4*np.pi*(bat_info['Dist_[Mpc]'][l250 == 0]*10**6*3.09e18)**2*(3.0e10/250.0e-4)*bat_herschel['PSW_err'][l250 == 0]*10**(-23)
l350[l350 == 0] = 4*np.pi*(bat_info['Dist_[Mpc]'][l350 == 0]*10**6*3.09e18)**2*(3.0e10/350.0e-4)*bat_herschel['PMW_err'][l350 == 0]*10**(-23)
l500[l500 == 0] = 4*np.pi*(bat_info['Dist_[Mpc]'][l500 == 0]*10**6*3.09e18)**2*(3.0e10/500.0e-4)*bat_herschel['PLW_err'][l500 == 0]*10**(-23)

In [11]:
# Create the censor vectors. TRUE indicates a detected point
cens250 = bat_herschel['PSW'] != 0
cens350 = bat_herschel['PMW'] != 0
cens500 = bat_herschel['PLW'] != 0

In [20]:
# Group the sample in to Sy 1, Sy 2, LINER, and AGN
bat_info['Sy_Type'] = pd.Series(index=bat_info.index)
for src in bat_info.index.values:

	type_split = bat_info.loc[src]['BAT_Type'].split()

	if (type_split[0] == 'Sy'):
		if ((type_split[1] == '1') | (type_split[1] == '1.2') | (type_split[1] == '1.4') | (type_split[1] == '1.5')):

			 bat_info.loc[src, 'Sy_Type'] = 'Sy 1'

		elif ((type_split[1] == '1.8') | (type_split[1] == '1.9') | (type_split[1] == '2')):
			bat_info.loc[src, 'Sy_Type']= 'Sy 2'
	else:

		if (type_split[0] == 'LINER'):
			bat_info.loc[src, 'Sy_Type']= 'LINER'

		elif (type_split[0] == 'AGN'):
			bat_info.loc[src, 'Sy_Type']= 'AGN'

In [24]:
# Create function to run the survival analysis
def calc_surv_mean(data, cens):
    # Convert arrays to R style arrays
    rdata = vecs.FloatVector(data)
    rcens = vecs.BoolVector(cens)

    mysurv = surv.Surv(rdata, rcens, type="left")
    fmla = Formula('y ~ 1')
    fmla.environment['y'] = mysurv
    mysurvfit = surv.survfit(fmla)
    
    summary = surv.summary_survfit(mysurvfit, rmean = True)
    
    return summary

In [32]:
# Run the survival analysis for 250 micron luminosities
surv_250_all = calc_surv_mean(np.log10(l250.values), cens250.values)
print('All 250 micron')
print(surv_250_all[-1])

All 250 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
313.00000000 313.00000000 313.00000000 313.00000000  42.75966727   0.03378735 
      median      0.95LCL      0.95UCL 
 42.86591633  42.80096694  42.92735256 



In [33]:
surv_250_sy1 = calc_surv_mean(np.log10(l250[bat_info.Sy_Type == 'Sy 1'].values), cens250[bat_info.Sy_Type=='Sy 1'].values)
print('Sy 1 250 micron')
print(surv_250_sy1[-1])

Sy 1 250 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
149.00000000 149.00000000 149.00000000 137.98814916  42.68949012   0.04003062 
      median      0.95LCL      0.95UCL 
 42.80096694  42.66787996  42.89906896 



In [34]:
surv_250_sy2 = calc_surv_mean(np.log10(l250[bat_info.Sy_Type == 'Sy 2'].values), cens250[bat_info.Sy_Type=='Sy 2'].values)
print('Sy 2 250 micron')
print(surv_250_sy2[-1])

Sy 2 250 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
157.00000000 157.00000000 157.00000000 157.00000000  42.83959847   0.04386191 
      median      0.95LCL      0.95UCL 
 42.89834708  42.84379339  43.01111939 



In [35]:
# Run the survival analysis for 350 micron luminosities
surv_350_all = calc_surv_mean(np.log10(l350.values), cens350.values)
print('All 350 micron')
print(surv_350_all[-1])

All 350 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
313.00000000 313.00000000 313.00000000 313.00000000  42.22038868   0.03670479 
      median      0.95LCL      0.95UCL 
 42.34044154  42.26026297  42.43537809 



In [36]:
surv_350_sy1 = calc_surv_mean(np.log10(l350[bat_info.Sy_Type == 'Sy 1'].values), cens350[bat_info.Sy_Type=='Sy 1'].values)
print('Sy 1 350 micron')
print(surv_350_sy1[-1])

Sy 1 350 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
149.00000000 149.00000000 149.00000000 129.05218916  42.17030174   0.03851256 
      median      0.95LCL      0.95UCL 
 42.24673273  42.08991321  42.42513544 



In [37]:
surv_350_sy2 = calc_surv_mean(np.log10(l350[bat_info.Sy_Type == 'Sy 2'].values), cens350[bat_info.Sy_Type=='Sy 2'].values)
print('Sy 2 350 micron')
print(surv_350_sy2[-1])

Sy 2 350 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
157.00000000 157.00000000 157.00000000 157.00000000  42.31835838   0.04586918 
      median      0.95LCL      0.95UCL 
 42.39155843  42.33963598  42.47461722 



In [40]:
# Run the survival analysis for 500 micron luminosities
surv_500_all = calc_surv_mean(np.log10(l500.values), cens500.values)
print('All 500 micron')
print(surv_500_all[-1])

All 500 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
313.00000000 313.00000000 313.00000000 280.28758539  41.59561243   0.02787581 
      median      0.95LCL      0.95UCL 
 41.66097777  41.59037196  41.73436038 



In [41]:
surv_500_sy1 = calc_surv_mean(np.log10(l500[bat_info.Sy_Type == 'Sy 1'].values), cens500[bat_info.Sy_Type=='Sy 1'].values)
print('Sy 1 500 micron')
print(surv_500_sy1[-1])

Sy 1 500 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
149.00000000 149.00000000 149.00000000 149.00000000  41.48862051   0.05379019 
      median      0.95LCL      0.95UCL 
 41.45121793  41.32708291  41.59404171 



In [42]:
surv_500_sy2 = calc_surv_mean(np.log10(l500[bat_info.Sy_Type == 'Sy 2'].values), cens500[bat_info.Sy_Type=='Sy 2'].values)
print('Sy 2 500 micron')
print(surv_500_sy2[-1])

Sy 2 500 micron
     records        n.max      n.start       events       *rmean   *se(rmean) 
157.00000000 157.00000000 157.00000000 143.63553862  41.69685967   0.03303504 
      median      0.95LCL      0.95UCL 
 41.73436038  41.66136375  41.85591047 

